## **Dependencies**

In [1]:
import numpy as np
import h5py
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
import cv2
from tensorflow.keras.layers import Input,Dense,Reshape,Conv2D,Dropout,multiply,Dot,Concatenate,subtract,ZeroPadding2D
from tensorflow.keras.layers import BatchNormalization,LeakyReLU,Flatten
from tensorflow.keras.layers import Conv2DTranspose as Deconv2d
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from keras import backend as K
from sklearn.utils import shuffle
import tensorflow as tf
import keras
from keras import layers
import os
from tqdm import tqdm
import re
from keras.preprocessing.image import img_to_array

# **Access the Dataset from Drive**

In [ ]:
from google.colab import drive
drive.mount('/content/drive')


In [ ]:
import os
folder_path = '/content/drive/MyDrive/archive/'
print(f"Contents of '{folder_path}': {os.listdir(folder_path)}")


In [ ]:
# To get the files in proper order

def sorted_alphanumeric(data):
    convert = lambda text: int(text) if text.isdigit() else text.lower()
    alphanum_key = lambda key: [convert(c) for c in re.split('([0-9]+)',key)]
    return sorted(data,key = alphanum_key)

# defining the size of the image
SIZE = 256
color_img = []
path = '/content/drive/MyDrive/archive/landscape Images/color'
files = os.listdir(path)
files = sorted_alphanumeric(files)
for i in tqdm(files):
        if i == '2200.jpg':       # Termination bcz there is 7000+ images
            break
        else:
            img = cv2.imread(path + '/'+i,1)

            # open cv reads images in BGR format so we have to convert it to RGB
            img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

            #resizing image
            img = cv2.resize(img, (SIZE, SIZE))
            img = img.astype('float32') / 255.0
            color_img.append(img_to_array(img))


gray_img = []
path = '/content/drive/MyDrive/archive/landscape Images/gray'
files = os.listdir(path)
files = sorted_alphanumeric(files)
for i in tqdm(files):
         if i == '2200.jpg':
            break
         else:
            img = cv2.imread(path + '/'+i,1)
            img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
            #resizing image
            img = cv2.resize(img, (SIZE, SIZE))
            img = img.astype('float32') / 255.0
            gray_img.append(img_to_array(img))

In [ ]:
#color_dataset=tf.data.Dataset.from_tensor_slices(np.array(color_img[:2000])).batch(64)
#gray_dataset=tf.data.Dataset.from_tensor_slices(np.array(gray_img[:2000])).batch(64)

#color_dataset_t=tf.data.Dataset.from_tensor_slices(np.array(color_img[2000:])).batch(8)
#gray_dataset_t=tf.data.Dataset.from_tensor_slices(np.array(gray_img[2000:])).batch(8)

In [ ]:
# Making another dataset from color_image and gray_image to train and test the data.

color_dataset = tf.data.Dataset.from_tensor_slices(np.array(color_img[:20])).batch(64)
gray_dataset = tf.data.Dataset.from_tensor_slices(np.array(gray_img[:20])).batch(64)

color_dataset_t = tf.data.Dataset.from_tensor_slices(np.array(color_img[20:40])).batch(8)
gray_dataset_t = tf.data.Dataset.from_tensor_slices(np.array(gray_img[20:40])).batch(8)


In [ ]:
# The code snippet you provided is using the iter() function and next() function to obtain the next batch of data from the color_dataset and gray_dataset.

example_color = next(iter(color_dataset))
example_gray = next(iter(gray_dataset))

In [ ]:
# The provided function plot_images is designed to visualize batches of color and grayscale images side by side.

def plot_images(a = 4):

    for i in range(a):
        plt.figure(figsize = (10,10))
        plt.subplot(121)
        plt.title('color')
        plt.imshow(example_color[i] )

        plt.subplot(122)
        plt.title('gray')
        plt.imshow(example_gray[i])
        plt.show()


In [ ]:

plot_images(4)

In [ ]:
#The provided code defines two functions, downsample and upsample, commonly used in the architecture of convolutional neural networks (CNNs),
# especially in the context of image processing tasks.
# These functions are typically employed in the construction of the encoder and decoder parts of a U-Net or similar architectures

def downsample(filters, size, apply_batchnorm=True):

  result = tf.keras.Sequential()
  result.add(tf.keras.layers.Conv2D(filters, size, strides=2, padding='same',kernel_initializer='he_normal', use_bias=False))

  if apply_batchnorm:
    result.add(tf.keras.layers.BatchNormalization())

  result.add(tf.keras.layers.LeakyReLU())
  return result

def upsample(filters, size, apply_dropout=False):

  result = tf.keras.Sequential()
  result.add(tf.keras.layers.Conv2DTranspose(filters, size, strides=2,padding='same',kernel_initializer='he_normal',use_bias=False))
  result.add(tf.keras.layers.BatchNormalization())

  if apply_dropout:
      result.add(tf.keras.layers.Dropout(0.5))

  result.add(tf.keras.layers.ReLU())
  return result

In [ ]:
# defining Generator

def Generator():
  inputs = tf.keras.layers.Input(shape=[256,256,3])

# Define the downsampling stack
  down_stack = [
    downsample(64, 4, apply_batchnorm=False), # (bs, 128, 128, 64)
    downsample(128, 4), # (bs, 64, 64, 128)
    downsample(256, 4), # (bs, 32, 32, 256)
    downsample(512, 4), # (bs, 16, 16, 512)
    downsample(512, 4), # (bs, 8, 8, 512)
    downsample(512, 4), # (bs, 4, 4, 512)
    downsample(512, 4), # (bs, 2, 2, 512)
    downsample(512, 4), # (bs, 1, 1, 512)
  ]

# Define the upsampling stack
  up_stack = [
    upsample(512, 4, apply_dropout=True), # (bs, 2, 2, 1024)
    upsample(512, 4, apply_dropout=True), # (bs, 4, 4, 1024)
    upsample(512, 4, apply_dropout=True), # (bs, 8, 8, 1024)
    upsample(512, 4), # (bs, 16, 16, 1024)
    upsample(256, 4), # (bs, 32, 32, 512)
    upsample(128, 4), # (bs, 64, 64, 256)
    upsample(64, 4), # (bs, 128, 128, 128)
  ]

# Initialize the last layer
  initializer = tf.random_normal_initializer(0., 0.02)
  last = tf.keras.layers.Conv2DTranspose(3, 4,strides=2,padding='same',kernel_initializer=initializer,activation='tanh') # (bs, 256, 256, 3)

  x = inputs

# Downsampling through the model
  skips = []
  for down in down_stack:
    x = down(x)
    skips.append(x)

  skips = reversed(skips[:-1])

# Upsampling and establishing the skip connections
  for up, skip in zip(up_stack, skips):
    x = up(x)
    x = tf.keras.layers.Concatenate()([x, skip])

# Final layer
  x = last(x)

  return tf.keras.Model(inputs=inputs, outputs=x)

In [ ]:
# Defining Discriminator

def Discriminator():

# Initialize weights using a random normal distribution
  initializer = tf.random_normal_initializer(0., 0.02)

# Define input layers for the real and target images
  inp = tf.keras.layers.Input(shape=[256, 256, 3], name='input_image')
  tar = tf.keras.layers.Input(shape=[256, 256, 3], name='target_image')

# Concatenate the input and target images along the channels axis
  x = tf.keras.layers.concatenate([inp, tar]) # (bs, 256, 256, channels*2)

# Downsample through the discriminator network
  down1 = downsample(64, 4, False)(x) # (bs, 128, 128, 64)
  down2 = downsample(128, 4)(down1) # (bs, 64, 64, 128)
  down3 = downsample(256, 4)(down2) # (bs, 32, 32, 256)

 # Apply zero-padding and convolutional layer
  zero_pad1 = tf.keras.layers.ZeroPadding2D()(down3) # (bs, 34, 34, 256)
  conv = tf.keras.layers.Conv2D(512, 4, strides=1,
                                kernel_initializer=initializer,
                                use_bias=False)(zero_pad1) # (bs, 31, 31, 512)

# Batch normalization and activation
  batchnorm1 = tf.keras.layers.BatchNormalization()(conv)
  leaky_relu = tf.keras.layers.LeakyReLU()(batchnorm1)

# Apply additional zero-padding and final convolutional layer
  zero_pad2 = tf.keras.layers.ZeroPadding2D()(leaky_relu) # (bs, 33, 33, 512)
  last = tf.keras.layers.Conv2D(1, 4, strides=1,
                                kernel_initializer=initializer)(zero_pad2) # (bs, 30, 30, 1)

# Create and return the discriminator model
  return tf.keras.Model(inputs=[inp, tar], outputs=last)

In [ ]:
generator = Generator()
generator.summary()

In [ ]:
discriminator = Discriminator()
discriminator.summary()

In [ ]:
# defined the loss functions and optimizers for your Generative Adversarial Network (GAN)
genLoss=[]
discLoss=[]

# Binary crossentropy los function is commonly used for GANs
loss_object = tf.keras.losses.BinaryCrossentropy(from_logits=True)

# Adam optimizer for updating the weights of the generator/discriminator.
generator_optimizer = tf.keras.optimizers.Adam(2e-4, beta_1=0.5)
discriminator_optimizer = tf.keras.optimizers.Adam(2e-4, beta_1=0.5)

# This constant is used to scale the L1 loss when calculating the total generator loss.
LAMBDA = 100

#This function calculates the total generator loss,
def generator_loss(disc_generated_output, gen_output, target):
  gan_loss = loss_object(tf.ones_like(disc_generated_output), disc_generated_output)

# mean absolute error
  l1_loss = tf.reduce_mean(tf.abs(target - gen_output))

  total_gen_loss = gan_loss + (LAMBDA * l1_loss)
  genLoss.append(total_gen_loss)

  return total_gen_loss, gan_loss, l1_loss

#This function calculates the total discriminator loss.
def discriminator_loss(disc_real_output, disc_generated_output):
  real_loss = loss_object(tf.ones_like(disc_real_output), disc_real_output)

  generated_loss = loss_object(tf.zeros_like(disc_generated_output), disc_generated_output)

  total_disc_loss = real_loss + generated_loss
  discLoss.append(total_disc_loss)

  return total_disc_loss

In [ ]:
# This function, train_step, defines a single training step for your Generative Adversarial Network (GAN)

def train_step(input_image, target, epoch):

 # Use GradientTape to record the operations for automatic differentiation
  with tf.GradientTape() as gen_tape, tf.GradientTape() as disc_tape:

 # Generate an output using the generator
    gen_output = generator(input_image, training=True)

 # Calculate the discriminator outputs for real and generated images
    disc_real_output = discriminator([input_image, target], training=True)
    disc_generated_output = discriminator([input_image, gen_output], training=True)

# Calculate generator and discriminator losses
    gen_total_loss, gen_gan_loss, gen_l1_loss = generator_loss(disc_generated_output, gen_output, target)
    disc_loss = discriminator_loss(disc_real_output, disc_generated_output)

 # Calculate gradients with respect to generator and discriminator variables
  generator_gradients = gen_tape.gradient(gen_total_loss,
                                          generator.trainable_variables)
  discriminator_gradients = disc_tape.gradient(disc_loss,
                                               discriminator.trainable_variables)

  # Apply gradients to update the weights of the generator and discriminator
  generator_optimizer.apply_gradients(zip(generator_gradients,
                                          generator.trainable_variables))
  discriminator_optimizer.apply_gradients(zip(discriminator_gradients,
                                              discriminator.trainable_variables))

In [ ]:
# allowing to monitor the progress of your model over multiple epochs

import time
def fit(train_ds, epochs,):
  for epoch in range(epochs):
    start = time.time()
    print("Epoch: ", epoch+1)

    # Train
    for n, (input_image, target) in train_ds.enumerate():
      train_step(input_image, target, epoch)
    print()
    print ('Time taken for epoch {} is {} sec\n'.format(epoch + 1,time.time()-start))

In [ ]:
# The generate_images function appears to be used for visualizing the results of your image colorization model

def generate_images(model, test_input, tar):

  # Generate predictions using the model
  prediction = model(test_input, training=True)

  # Create a figure for visualization
  plt.figure(figsize=(15,15))

# Prepare images and titles for display
  display_list = [test_input[0], tar[0], prediction[0]]
  title = ['Input Image', 'Ground Truth', 'Predicted Image']

  # Display the images and titles in a subplot
  for i in range(3):
    plt.subplot(1, 3, i+1)
    plt.title(title[i])

    # getting the pixel values between [0, 1] to plot it.
    plt.imshow(display_list[i])
    plt.axis('off')

     # Show the figure with the generated images
  plt.show()

# Generate and display images for two examples from the datasets
for example_input, example_target in tf.data.Dataset.zip((gray_dataset,color_dataset)).take(2):
  generate_images(generator, example_input, example_target)


In [ ]:
hist=fit(tf.data.Dataset.zip((gray_dataset, color_dataset)), epochs = 10)

In [ ]:
#  visualize the training progress of a GAN (Generative Adversarial Network).

import seaborn as sns
sns.set(style='whitegrid')
plt.figure(figsize=(10,4))
plt.plot(genLoss, label="Generator Loss")
plt.plot(discLoss, label="Discriminator Loss")
plt.title("GAN LOSS VALUES")
plt.xlabel("Iterations")
plt.ylabel("Loss")
plt.legend()
plt.show()

In [ ]:
def generate_images(model, test_input, tar):
    # Generate predictions using the model
    prediction = model(test_input, training=True)

    # Set up the figure for displaying images
    plt.figure(figsize=(15, 15))

    # Prepare the images and titles for display
    display_list = [test_input[0], tar[0], prediction[0]]
    title = ['Input Image', 'Ground Truth', 'Predicted Image']

    # Iterate over the images and titles, and display them
    for i in range(3):
        plt.subplot(1, 3, i + 1)
        plt.title(title[i])

        # Clip pixel values to the valid range [0, 1]
        img = np.clip(display_list[i], 0, 1)

        # Getting the pixel values between [0, 1] to plot it.
        plt.imshow(img)
        plt.axis('off')

    # Show the figure with the generated images
    plt.show()

# Set up the figure outside the loop
plt.figure(figsize=(15, 15))

# Iterate over 5 examples and generate/display images
for example_input, example_target in tf.data.Dataset.zip((gray_dataset, color_dataset)).take(5):
    generate_images(generator, example_input, example_target)
